### Realizando todos os imports que serão necessário para o equacionamento do TG

In [93]:
import sympy  as sp
import numpy as np 
import pandas as pd 
import math as mt 
import matplotlib as plt

In [204]:
x, y, z, phi, T, T_max, T1, T2, T4,T_mr,  delta, P1, P2, P3, A_ref, A_h_eff, phi_ZP, theta, q_ref, A_h_reff, delta_P_3_4, R, m_ponto_3, T3, k, D_ref, phi_global, m_ponto_zp,  = sp.symbols(
    [
    'x','y','z', 'phi', 'T', 'T_max', 'T1', 'T2', 'T4', 'T_mr', 'delta', 'P1',
    'P2', 'P3', 'A_ref', 'A_h_eff', 'phi_ZP', 'theta', 'q_ref', 'A_h_reff', 
    'delta_P_3_4','R', 'm_ponto_3','T3', 'k', 'D_ref', 'phi_global', 'm_ponto_zp'
    ])




𝑚̇

### Funcao da equação (5):

𝐶𝑥𝐻𝑦 + 𝑎𝑂2 → 𝑥𝐶𝑂2 + (𝑦⁄2)𝐻2O

In [192]:
 
def razao_ar_comb_esteq(a, PMar, PMcomb):
    A_C_esteq = a*(1 + 3.7274 + 0.0458) * (PMar/PMcomb)
    return A_C_esteq

teste = razao_ar_comb_esteq(17.75, 28.9645, 167.3087)
# phi_esteq = 1/teste
# phi_esteq


### razão de equivalência
A razão de equivalência é uma relação frequentemente usada para indicar se a mistura
está pobre, rica ou está estequiométrica. A razão de equivalência é definida como:

𝜙 =
(𝐴/𝐶)𝑒𝑠𝑡𝑒𝑞
(𝐴/𝐶)𝑟𝑒𝑎𝑙
=
(𝐶/𝐴)𝑟𝑒𝑎𝑙
(𝐶/𝐴)𝑒𝑠𝑡𝑒q

In [193]:
def razao_equivalencia(A_C_esteq, A_C_real):
    phi = A_C_esteq/A_C_real
    return ('rica' if phi > 1 else 'pobre')




DUVIDA : COMO FAREMOS COM RELAÇÃO AO EQUACIONAMENTO DA Temperatura de Chama Adiabática ? 

### Turbina a gás

- Componente 1:
    - Compressor, que recebe potência de eixo da turbina, converte em energia que é
entregue ao ar elevando a sua entalpia e, por consequência, a pressão e a temperatura
desse fluido de trabalho

- Componente 2:
    - Câmara de combustão, onde o ar, proveniente do compressor, é misturado ao
combustível, formando os reagentes. É fornecida energia de ativação para que a
combustão desses reagentes inicie e se processe em seu interior


- Componente 3:
    - Turbina, dispositivo mecânico ideal que converte a entalpia dos gases quentes advindos
da câmara de combustão em energia mecânica em potência de eixo

### Perda de pressao quente/fria

- Sendo 𝑃1 e 𝑃2 as pressões totais nas seções transversais logo ao início e logo ao final do difusor, respectivamente

- Onde os parâmetros 𝐴𝑟𝑒𝑓 e 𝑞𝑟𝑒𝑓 são, respectivamente, a área transversal e a pressão
dinâmica avaliadas num plano de máxima área transversal da câmara de combustão, delimitada
pelo seu invólucro

- Já o termo 𝐴ℎ,𝑒𝑓𝑓, que representa a área efetiva total dos orifícios do tubo
de chama, pode ser calculado de duas formas

In [194]:
def perda_pressao_quente(rho, u, T4, T3):
    delta_P_quente = 0.5 * rho * (u**2) * ((T4/T3) - 1 )
    return delta_P_quente



delta_P_difusor = P1 - P2  #Sendo 𝑃1 e 𝑃2 as pressões totais nas seções transversais logo ao início e logo ao final do difusor, respectivamente

def P_turboChama(A_ref, A_h_eff, q_ref):   

    delta_P_turbo_chama = ((A_ref/A_h_eff) ** 2) * q_ref

    return delta_P_turbo_chama

def perda_pressao_fria(P_difusor, P_turbo_chama):
    delta_P_fria = sum(P_difusor, P_turbo_chama)
    return delta_P_fria


### Dimensionalizando os orificios do tubo de chama:

In [195]:
def dimensao_orificio_tubo_chama(A_ref, delta_p_3_4, q_ref, delta_P_difusor):
    dimensao_orificioTuboChama = A_ref/(((delta_p_3_4/q_ref) - (delta_P_difusor/q_ref))**0.5)
    return dimensao_orificioTuboChama

### Calculando A_h_reff

In [196]:
#Como esperando muitos valores de entrada para fazermos o somatório, passaremos como entrada uma lista

C_d = []
A_h = []

def A_h_eff_function(C_d, A_h):
    somatorio_A_h_reff = 0
    for c in range(len(C_d)):
        somatorio_A_h_reff =  somatorio_A_h_reff + C_d[c] * A_h[c]  
        
    return somatorio_A_h_reff

# A_h_eff_function(C_d, A_h)


### Parametros para especificar a perda de pressão

- O primeiro (∆𝑃3−4/𝑃3), chamado de perda de pressão global, geralmente assume
valores entre 4% a 8%, dependendo do tipo de câmara de combustão. O segundo termo
(∆𝑃3−4/𝑞𝑟𝑒𝑓), chamado de fator de perda de pressão, é mais expressivo para o projetista pois
denota a resistência do fluido em atravessar a câmara de combustão. Aerodinamicamente, pode
ser considerado um equivalente ao “coeficiente de arrasto”. Ambos os termos não consideram
a perda “quente” 

In [197]:
delta_P_3_4 = P3 * (delta_P_3_4/q_ref) * (R/2) * ( (((m_ponto_3 * T3)**0.5)/ (A_ref*P3))**2 )

delta_P_3_4

R*delta_P_3_4*(T3*m_ponto_3)**1.0/(2*A_ref**2*P3*q_ref)

### Qualidade Transversal de temp e Perfil Radial de temp


- São dois os termos usados para descrever o perfil de temperatura transversal: a
Qualidade Transversal de Temperatura (em inglês, Temperature Factor, Traverse Number ou
Pattern Factor) e o Perfil Radial de Temperatura (em inglês, Profile Factor), expressos
matematicamente a seguir:

In [198]:
def qualidade_tranversal_temp(T_max, T4, T3):
    qualidade_trans_temp = (T_max - T4)/(T4 - T3)
    return qualidade_trans_temp

def perfil_radial_temp(T_mr, T4, T3):
    perfil_rad_temp = (T_mr - T4)/(T4 - T3)
    return perfil_rad_temp

#Onde 𝑇𝑚𝑎𝑥 é a máxima temperatura dos gases na saída da câmara de combustão e 𝑇𝑚𝑟 é a temperatura média mais alta de uma linha de circunferência escolhida
    

### Informacões para dimensionar do combustor

- Vazão mássica
- Vazão mássica de combustível
- Distribuição de temperatura, pressão e velocidade na entrada do combustor;
- Perfil transversal de temperatura na saída do combustor;
- Limitações de perda de pressão;
- Tipos de poluentes e níveis permitidos
- Temperatura na parede permitida
- Limitações de tamanho
- Limitações de peso
- Requisitos de durabilidade e semelhança de vida útil entre as trocas de peças;
- Compatibilidade com o tipo de combustível;
- Outras limitações ou requisitos especiais.


### Calculo da área de transfência

- A constante 𝑘 varia apenas com relação às unidades usadas, é igual à 143,5 ao utilizar
a unidades do sistema internacional (SI) e é igual à 0,83, nas unidades 𝑙𝑏, 𝑓𝑡 𝑒 𝑅. O termo 𝑚̇3
é vazão mássica de ar, 𝑃3 é a pressão total e 𝑇3 a temperatura estimadas na entrada da câmara,
𝑞𝑟𝑒𝑓 é a pressão dinâmica na máxima área da seção transversal da carcaça (𝐴𝑟𝑒𝑓) e ∆𝑃3−4 é a
perda de pressão total ao longo da câmara.
Já  é a variável que se relaciona com a eficiência da combustão, . Como se busca
obter a maior eficiência possível, é utilizado o parâmetro  correspondente, tendo seu valor
igual à 73 . 106
(unidades do SI)

- O fator de correção da temperatura, b, é definido de acordo com a relação ar combustível
da zona primária. Para razão de equivalência nesta região entre 0,6 e 1,0, a fórmula 𝑏 =
245 (1,39 + ln 
𝑍𝑃) é empregada. Já para 
𝑍𝑃 entre 1,0 e 1,4

- A razão 𝑚̇𝑍𝑃 𝑚̇ ⁄ 3
, ou melhor, a porcentagem de ar a ser designada à zona primária, deve
ser escolhida de modo que a razão de equivalência no local esteja na região de flamabilidade
da mistura

In [199]:

def area_tranferencia_aerodinamica(k, m_ponto_3m, T3, P3, delta_P_3_4, q_ref):
    area_transf = k * ( ((m_ponto_3 * (T3**0.5) )/(P3))**2  * ((delta_P_3_4/q_ref)/(delta_P_3_4/P3)) ** 0.5 )
    return area_transf

def area_tranferencia_quimica(P3, A_ref, D_ref, T3, b, m_ponto_3):
    theta = ((P3**1.75) * A_ref * (D_ref ** 0.75) * np.exp(T3/b))/ m_ponto_3
    return theta



In [202]:

def phi_global_function(m_comb_op, m_ponto_3_op, m_comb, m_ponto_3):
    phi_global = (m_comb_op/m_ponto_3_op)/ (m_comb/m_ponto_3)

def phi_zp_function(phi_global, m_ponto_zp, m_ponto_3):
    phi_ZP = phi_global/(m_ponto_zp/m_ponto_3)
    return phi_ZP